In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 35 kB/s 
     |████████████████████████████████| 198 kB 17.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=f27ea8b1dc9d2d8291c60e377091a3a51598e6c67e2e8155e79e5bf5fb661732
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('Home_work_spark')\
        .getOrCreate()
spark

In [53]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StructType,StructField, StringType, IntegerType , BooleanType,DateType

# schema = StructType() \
#       .add("new_cases",IntegerType(),True) \
#       .add("total_cases",IntegerType(),True) \
#       .add("iso_code",StringType(),True) \
#       .add("location",StringType(),True) \
#       .add("population",IntegerType(),True) \
#       .add("date", DateType(), True)

      
df = spark.read.csv('owid-covid-data.csv',header=True).select(
        f.col("date"),
        f.col("total_cases").cast("int"),
        f.col("new_cases").cast("int"),  
        f.col("iso_code").cast("string"), 
        f.col("location").cast("string"),
        f.col("population").cast("int"))

df.show(1)

df.printSchema()

+----------+-----------+---------+--------+-----------+----------+
|      date|total_cases|new_cases|iso_code|   location|population|
+----------+-----------+---------+--------+-----------+----------+
|2020-02-24|          1|        1|     AFG|Afghanistan|  38928341|
+----------+-----------+---------+--------+-----------+----------+
only showing top 1 row

root
 |-- date: string (nullable = true)
 |-- total_cases: integer (nullable = true)
 |-- new_cases: integer (nullable = true)
 |-- iso_code: string (nullable = true)
 |-- location: string (nullable = true)
 |-- population: integer (nullable = true)



In [54]:
#Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

# вижу что есть какие-то WORLD и тд, фильтрую на 3 строке коды, т к страны состоят из 3 букв

df.filter(f.col('date') == '2021-03-31').\
  filter(f.length(f.col('iso_code')) == 3).\
  select('iso_code','total_cases','location','population').\
  withColumn('persent', 
             df.total_cases / df.population * 100).\
  sort(f.col('persent').desc()).\
  select('iso_code','total_cases','persent').limit(15).show()
  
  #.repartition(1).write.csv("task1.1", header = True)


+--------+-----------+------------------+
|iso_code|total_cases|           persent|
+--------+-----------+------------------+
|     AND|      12010|15.543907331909661|
|     MNE|      91218|14.523725364693293|
|     CZE|    1532332|14.308848404077997|
|     SMR|       4730|13.937179562732041|
|     SVN|     215602|10.370805779121204|
|     LUX|      61642| 9.847342390123583|
|     ISR|     833105| 9.625106044786802|
|     USA|   30462210| 9.203010995860707|
|     SRB|     600596| 8.826328557933492|
|     BHR|     144445| 8.488860079114566|
|     PAN|     355051| 8.228739065460761|
|     PRT|     821722| 8.058699735120369|
|     EST|     106424| 8.022681579659551|
|     SWE|     804886| 7.969744347858805|
|     LTU|     216119| 7.938864728274825|
+--------+-----------+------------------+



In [57]:
#Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
#(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)


# фильтрую по датам, фильтрую чтобы были только страны с кодом в 3 символа, нахожу max в каждой локации за неделю
# беру первые 10 стран из топа, джоин чтобы получить все строки для каждой из этих стран 

gff = df.filter('date between "2021-03-24" and "2021-03-31" and length(iso_code) = 3')
gff.\
groupBy('location').agg(f.max("new_cases").alias("new_cases")).\
sort(f.col('new_cases').desc()).\
limit(10).select('location','new_cases').\
join(gff, on = ['location', 'new_cases']).\
sort(f.col("new_cases").desc()).\
select('date','location','new_cases').show()

#.repartition(1).write.csv("task1.2", header = True)


In [56]:
#Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. 
#(например: в россии вчера было 9150 , сегодня 8763, итог: -387) 
#(в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)
from pyspark.sql import Window

df.filter('iso_code = "RUS"').filter('date between "2021-03-24" and "2021-03-31"').\
withColumn('before', f.lag(f.col('new_cases')).over(Window.orderBy('date'))).\
withColumn('diff', f.col('new_cases') - f.col ('before')).\
select('date','before','new_cases','diff').\
orderBy('date').show()

#.repartition(1).write.csv("task1.3", header = True)

+----------+------+---------+----+
|      date|before|new_cases|diff|
+----------+------+---------+----+
|2021-03-24|  null|     8769|null|
|2021-03-25|  8769|     9128| 359|
|2021-03-26|  9128|     9073| -55|
|2021-03-27|  9073|     8783|-290|
|2021-03-28|  8783|     8979| 196|
|2021-03-29|  8979|     8589|-390|
|2021-03-30|  8589|     8162|-427|
|2021-03-31|  8162|     8156|  -6|
+----------+------+---------+----+

